In [1]:
%load_ext autoreload
%autoreload 2

import os

import tqdm
from rich import print
from rich.traceback import install

import datasets

os.environ["HYDRA_FULL_ERROR"] = "1"
install()

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7f5f8ae9c220>>

In [2]:
# ! pip install -e ../
# ! mamba install -c conda-forge itables pandas -y
# ! pip install pytorchvideo
import copy
import json
import os
import pathlib
import random
import sys
import time
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from dataclasses import dataclass
from enum import Enum
from math import floor
from typing import Any, Callable, Dict, List, Optional, Tuple, Union
import datasets

import numpy as np
import pandas as pd
import PIL
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import torch
import torch.nn as nn
import tqdm
from hydra_zen import builds, instantiate
from PIL import Image
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
from rich import print
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataloader import default_collate
from torchvision.transforms import Compose, RandomCrop, Resize, ToTensor
from torchvision.transforms._transforms_video import CenterCropVideo
from traitlets import default
from transformers import CLIPModel, CLIPProcessor
import datasets
from tali_wit.data import (
    AnyModalSample,
    dataclass_collate,
    default_image_transforms,
    ModalityTypes,
    select_subtitles_between_timestamps,
    TALIDataset,
)
from tqdm.notebook import tqdm



/home/evolvingfungus/miniconda/envs/minimal-ml-template/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/evolvingfungus/miniconda/envs/minimal-ml-template/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [3]:
from tali_wit.decorators import configurable
from tali_wit.utils import get_logger, load_json, save_json
from tali_wit.models import ModalityConfig
import gradio as gr

logger = get_logger(__name__)

def update_modality_panel(modality_type):
    if modality_type == "text":
        gr.update(children=[gr.inputs.Textbox(label="Text")])
    elif modality_type == "image":
        gr.update(children=[gr.inputs.Image(label="Image")])
    elif modality_type == "video":
        gr.update(children=[gr.inputs.Video(label="Video")])
    elif modality_type == "audio":
        gr.update(children=[gr.inputs.Audio(label="Audio")])
    else:
        raise ValueError(f"Unknown modality type: {modality_type}")
    

In [13]:

callback = gr.CSVLogger()

num_samples = 100

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    source = gr.Markdown("### Input Source", interactive=False)
    description_source_modality = gr.Markdown("Select the type of input modality you want to use to rank the results from the tabs", interactive=False)
    description_source_type = gr.Markdown("Once you do, next, choose the way you wish to input the data. You can either upload a file, use your webcam, or draw on the canvas.", interactive=False)
    description_rank = gr.Markdown("Once you have selected a source modality, then selected an input source, and finally uploaded your data, click the Rank button", interactive=False)
    
    with gr.Row():
        with gr.Column():
            with gr.Tab("text"):
                with gr.Tab("generic"):
                    generic_text = gr.Textbox(label="Text")
                    rank_by_generic_text_button = gr.Button("Rank Results", label="Rank Results")
                with gr.Tab("caption"):
                    caption_text = gr.Textbox(label="Caption")
                    rank_by_caption_text_button = gr.Button("Rank Results", label="Rank Results")
                with gr.Tab("title"):
                    title_text = gr.Textbox(label="Title")
                    rank_by_title_text_button = gr.Button("Rank Results", label="Rank Results")
                with gr.Tab("description"):
                    description_text = gr.Textbox(label="Description")
                    rank_by_description_text_button = gr.Button("Rank Results", label="Rank Results")
                with gr.Tab("subtitles"):
                    subtitles_text = gr.Textbox(label="Subtitles")
                    rank_by_subtitles_text_button = gr.Button("Rank Results", label="Rank Results")
                    
            with gr.Tab("image"):
                with gr.Tab("upload-image"):
                    uploaded_image = gr.Image(label="Image")
                    rank_by_uploaded_image_button = gr.Button("Rank Results", label="Rank Results")
                with gr.Tab("camera"):
                    recorded_image = gr.Image(source="webcam", streaming=True)
                    rank_by_recorded_image_button = gr.Button("Rank Results", label="Rank Results")
                with gr.Tab("draw"):
                    drawn_image = gr.Image(source="canvas")
                    rank_by_drawn_image_button = gr.Button("Rank Results", label="Rank Results")
                    
            with gr.Tab("video"):
                with gr.Tab("upload-video"):
                    upload_video = gr.Video(label="Video")
                    rank_by_upload_video_button = gr.Button("Rank Results", label="Rank Results")
                with gr.Tab("camera"):
                    camera_video = gr.Video(source="webcam")
                    rank_by_camera_video_button = gr.Button("Rank Results", label="Rank Results")
                
            with gr.Tab("audio"):
                with gr.Tab("upload-audio"):
                    upload_audio = gr.Audio(label="Audio")
                    rank_by_audio_upload_button = gr.Button("Rank Results", label="Rank Results")
                with gr.Tab("microphone"):
                    mic_audio = gr.Audio(source="microphone")
                    rank_by_audio_mic_button = gr.Button("Rank Results", label="Rank Results")
                
               
    images = {'wit-images': [], 'tali-images': []}
    videos = {'tali-videos': []}
    audio = {'tali-audio': []}
    text = {'wit-caption': [], 'wit-title': [], 'wit-description': [], 'wit-subtitles': [], 'tali-caption': []}
    
    source_content = gr.Markdown("Fetch some samples to rank, by clicking the button below! 🖼️ 🎦 🔊 📝", interactive=False)
    
    fetch_random_samples_button = gr.Button("Fetch Random TALI Samples", label="Fetch Random Samples")
    
    with gr.Row():
        with gr.Column():
           for key in images:
               with gr.Tab(key):
                   for i in range(num_samples):
                       images[key].append(gr.Image())
            
        with gr.Column():
            for key in videos:
                with gr.Tab(key):
                    for i in range(num_samples):
                        videos[key].append(gr.Video())
        
        with gr.Column():
            for key in audio:
                with gr.Tab(key):
                    for i in range(num_samples):
                        audio[key].append(gr.Audio())
        
        with gr.Column():
            for key in text:
                with gr.Tab(key):
                    for i in range(num_samples):
                        text[key].append(gr.Textbox())
            
            
            # input_modality.change(update_modality_panel, [input_modality], [modality_panel])

    
            
            
   

In [14]:
demo.queue(concurrency_count=8)
demo.launch(share=True, debug=True, enable_queue=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://c85a63ac00ca715018.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://c85a63ac00ca715018.gradio.live
